In [200]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [211]:
class LIFEstate(object):
    def __init__(self, starting_locations):
        self.locations = starting_locations
        
    def get_neighbors(self, location):
        for n , m in itertools.product([-1, 0, 1], [-1, 0, 1]):
            if  (n == 0 and m == 0):
                continue
            yield location[0] + n, location[1] + m
            
    def get_all_locations(self, active_locations):
        nearby_locations = dict((k,1) for k in active_locations)
        for location in active_locations:  
            for neighbor in self.get_neighbors(location):
                if (neighbor in active_locations):
                    continue
                nearby_locations[neighbor] = 0
            
        all_locations = nearby_locations.copy()
        for location in nearby_locations:
            if(nearby_locations[location] == 1):
                continue
            for neighbor in self.get_neighbors(location):
                if(neighbor in nearby_locations):
                    continue
                all_locations[neighbor] = -1
        return all_locations    

    def evaluate_next_generation(self, all_locations):
        new_active_locations = {}
        for location in all_locations:
            if(all_locations[location] == -1):
                continue
            if(all_locations[location] == 0):
                count = 0
                for neighbor in self.get_neighbors(location):
                    if(all_locations[neighbor] == 1):
                        count += 1
                if(count == 3):
                    new_active_locations[location] = 1
            if(all_locations[location] == 1):
                count = 0
                for neighbor in self.get_neighbors(location):
                    if(all_locations[neighbor] == 1):
                        count += 1
                if(count == 2 or count == 3):
                    new_active_locations[location] = 1
        return list(new_active_locations.keys())
    
    def get_next_generation(self):
        all_locations = self.get_all_locations(self.locations)
        new_active_locations = self.evaluate_next_generation(all_locations)
        self.locations = new_active_locations
        return self.locations

In [212]:
class Automata(object):
    def __init__(self, ax, LIFEstate):
        self.lifestate = LIFEstate
        self.points = LIFEstate.locations
        self.frame, = ax.plot([],[], 'o', markersize = 17) #why on earth does this need a comma to go 
        self.ax = ax
        
        #plot parameters
        self.padding = 3
        x = np.array(self.points, (int,int)).transpose()
        self.xmin = np.min(x[0]) - self.padding
        self.xmax = np.max(x[0]) + self.padding
        self.ymin = np.min(x[1]) - self.padding
        self.ymax = np.max(x[1]) + self.padding
        
        self.ax.set_xlim(self.xmin, self.xmax)
        self.ax.set_ylim(self.ymin, self.ymax)
        
    def init(self):
        x = np.array(self.points, (int,int)).transpose()
        self.frame.set_data(x[0], x[1])
        return self.frame,
    
    def __call__(self, i):
        if i == 0:
            return self.init()
        
        self.points = self.state.get_next_generation()
        if self.points == []:
            self.frame.set_data([],[])
            return self.frame,     
        
        x = np.array(self.points, (int,int)).transpose()
        
        self.xmin = np.min(x[0]) - self.padding
        self.xmax = np.max(x[0]) + self.padding
        self.ymin = np.min(x[1]) - self.padding
        self.ymax = np.max(x[1]) + self.padding
        
        self.ax.set_xlim(self.xmin, self.xmax)
        self.ax.set_ylim(self.ymin, self.ymax)
        self.frame.set_data(x[0],x[1])
        
        return self.frame,


In [213]:
fig, ax = plt.subplots()
lifestate = LIFEstate([(0,0), (0,1), (1,1), (0,2), (2,2), (3,3), (4,4), (5,5)])
a = Automata(ax, lifestate)
anim = FuncAnimation(fig, a, frames=np.arange(100), init_func=a.init,
                     interval=100, blit=True)
plt.show()

In [ ]:
# [(0,0), (0,1), (1,1), (0,2), (2,2), (3,3), (4,4), (5,5)]
#[(0,0), (0,1), (0,2),(0,3), (0,4), (0,5),(1,0), (1,1), (1,2),(1,3), (1,4), (1,5), (2,0), (2,1), (2,2),(2,3), (2,4), (2,5)]